In [49]:
p1 = 4  # test
p2 = 8  # test

p1 = 6
p2 = 7

In [50]:
# Modulo but from 1 to m.
# I did this in previous assignments with an if-statement, but I recently learned this version is better
# From stackoverflow
nmod = lambda x, m: (x-1) % m + 1

def triang(n, m):
    if n+m > 101:
        # e.g. triang(98, 5) = triang(98, 3) + triang(1, 2)
        return triang(n, 101-n) + triang(1, m-101+n)

    # Calculates m triangular numbers starting from n
    # e.g. triang(22, 3) = 22+23+24 = 69. Nice
    # if t(n) is triangular numbers, this is equal to
    # t(n+m-1) - t(n-1) = (n+m-1)(n+m)/2 - (n-1)(n)/2 = (n+m-1)(n+m)-(n-1)(n) / 2
    # which simplifies to:
    return (m*m + 2*m*n - m) / 2


In [51]:
def roll(position, dice):
    # Rolls 3 times, then returns new position, dice
    return nmod(position+triang(dice, 3), 10), dice+3

def play(p1, p2, dice=1):
    # returns p1_score, p2_score, turns
    # Where turns is the amount of times the dice had been thrown when the game is over
    p1_score, p2_score, turns = 0, 0, 0
    while p1_score < 1000 and p2_score < 1000:
        # Player 1
        p1, dice = roll(p1, dice)
        p1_score += p1
        turns += 3
        if p1_score >= 1000: break
        # Player 2
        p2, dice = roll(p2, dice)
        p2_score += p2
        turns += 3
    return p1_score, p2_score, turns

p1_score, p2_score, turns = play(p1, p2)
int(min(p1_score, p2_score) * turns)


921585

In [116]:
# Naive (exponential) implementation: we will almost certainly need to make some lookup table
# To store known results in. Buit first make it naively

tadd = lambda x, y, z: (x[0] + y[0] + z[0], x[1] + y[1] + z[1])

def dirac_play(p1, p2, p1_score=0, p2_score=0, p1_turn=True, subrolls=0, subresult=0, m=21):
    # print(f"p1{p1}, p2{p2}, s1{p1_score}, s2{p2_score}, t{p1_turn}, r{subrolls}, s{subresult}")
    # returns p1_wins, p2_wins, which is the sum of games either players wins from this position
    if subrolls == 0 and p1_score >= m: return 1, 0
    if subrolls == 0 and p2_score >= m: return 0, 1
    if (p1, p2, p1_score, p2_score, p1_turn, subrolls, subresult) in known_positions:
        return known_positions[(p1, p2, p1_score, p2_score, p1_turn, subrolls, subresult)]
    
    if subrolls == 3:
        # Switch turn
        if p1_turn:
            return dirac_play(nmod(p1+subresult, 10), p2, p1_score+nmod(p1+subresult, 10), p2_score, False, 0, 0, m)
        else:
            return dirac_play(p1, nmod(p2+subresult, 10), p1_score, p2_score+nmod(p2+subresult, 10), True, 0, 0, m)

    if p1_turn:
        result = tadd(
            dirac_play(p1, p2, p1_score, p2_score, p1_turn, subrolls+1, subresult+1, m),
            dirac_play(p1, p2, p1_score, p2_score, p1_turn, subrolls+1, subresult+2, m),
            dirac_play(p1, p2, p1_score, p2_score, p1_turn, subrolls+1, subresult+3, m)
        )
    else:
        result = tadd(
            dirac_play(p1, p2, p1_score, p2_score, p1_turn, subrolls+1, subresult+1, m),
            dirac_play(p1, p2, p1_score, p2_score, p1_turn, subrolls+1, subresult+2, m),
            dirac_play(p1, p2, p1_score, p2_score, p1_turn, subrolls+1, subresult+3, m)
        )
    known_positions[(p1, p2, p1_score, p2_score, p1_turn, subrolls, subresult)] = result
    return result
        


In [119]:
known_positions = {}  # Global variable. Reset it everytime before running calculation

max(dirac_play(p1, p2))

911090395997650